In [1]:
# ==========================
# Cell 1: Imports & Setup
# ==========================
import os
import io
import joblib
import threading
import pandas as pd
import numpy as np
import sounddevice as sd
import scipy.io.wavfile as wavfile
import spacy
from googletrans import Translator
import speech_recognition as sr

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# optional / heavy deps (wrapped later)
try:
    from speechbrain.pretrained import EncoderClassifier
    SPEECHBRAIN_AVAILABLE = True
except Exception:
    SPEECHBRAIN_AVAILABLE = False

try:
    import torchaudio
except Exception:
    torchaudio = None

# Fallback sentiment + acoustic features
try:
    from transformers import pipeline
    TRANSFORMERS_AVAILABLE = True
except Exception:
    TRANSFORMERS_AVAILABLE = False

try:
    import librosa
except Exception:
    librosa = None

# Language model for text preprocessing
nlp = spacy.load("en_core_web_sm")

# File names (non-hardcoded dataset names used, user can change)
SYMPTOM_FILE = "Disease_symptoms_new.csv"    # should have columns: Disease,Symptoms  (order tolerant)
TEST_FILE = "Disease_Tests.csv"              # should have column 'Disease' and other test columns
INTENSITY_FILE = "Disease_Intensity.csv"     # should have columns ['Disease','Intensity'] or similar

MODEL_FILE = "disease_model.pkl"
VECTORIZER_FILE = "vectorizer.pkl"

# A small keywords list to help with symptom extraction if needed
DEFAULT_SYMPTOMS = [
    "fever","cough","headache","fatigue","nausea","vomiting","dizziness","chest pain",
    "shortness of breath","sore throat","diarrhea","rash","body ache","cold","congestion",
    "chills","loss of taste","loss of smell","runny nose","muscle pain"
]


c:\Users\klvns\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\klvns\AppData\Local\Programs\Python\Python311\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
c:\Users\klvns\AppData\Local\Programs\Python\Python311\Lib\inspect.py:992: UserWarning: Module 'speechbrain.pretrained' was deprecate

In [2]:
# ==========================
# Cell: Translation Utils
# ==========================
from langdetect import detect
from googletrans import Translator

translator = Translator()

def ensure_english(text: str) -> str:
    """Detects language and translates to English if needed."""
    try:
        lang = detect(text)
    except Exception:
        lang = "unknown"
    if lang != "en":
        try:
            translated = translator.translate(text, src=lang, dest="en").text
            print(f"🌐 Translated from {lang} → en: {translated}")
            return translated
        except Exception as e:
            print("⚠️ Translation failed, using original text:", e)
            return text
    return text


In [3]:
# ==========================
# Cell 2: Preprocess & Utilities
# ==========================
def preprocess(text):
    """Light lemmatization + stopword/punct removal using spaCy."""
    doc = nlp(str(text).lower())
    return " ".join([token.lemma_ for token in doc if token.is_alpha and not token.is_stop])

def ensure_dataframe_columns(df, expected_cols):
    """Return dataframe with expected columns (case-insensitive)."""
    cols_lower = {c.lower(): c for c in df.columns}
    mapping = {}
    for col in expected_cols:
        if col.lower() in cols_lower:
            mapping[cols_lower[col.lower()]] = col
    if mapping:
        df = df.rename(columns=mapping)
    return df


In [4]:
# ==========================
# Cell 3: SymptomAgent Class (learning agent)
# ==========================
class SymptomAgent:
    def __init__(self, symptom_file=SYMPTOM_FILE, test_file=TEST_FILE, intensity_file=INTENSITY_FILE):
        self.symptom_file = symptom_file
        self.test_file = test_file
        self.intensity_file = intensity_file

        # load datasets (will raise if not present)
        self.df_symptoms = pd.read_csv(self.symptom_file)
        self.df_symptoms = ensure_dataframe_columns(self.df_symptoms, ["Disease", "Symptoms"])
        # ensure columns exist
        if "Disease" not in self.df_symptoms.columns or "Symptoms" not in self.df_symptoms.columns:
            raise ValueError(f"{self.symptom_file} must contain columns 'Disease' and 'Symptoms'")

        if os.path.exists(self.test_file):
            self.df_tests = pd.read_csv(self.test_file)
            self.df_tests = ensure_dataframe_columns(self.df_tests, ["Disease"])
        else:
            self.df_tests = pd.DataFrame(columns=["Disease"])

        if os.path.exists(self.intensity_file):
            self.df_intensity = pd.read_csv(self.intensity_file)
            self.df_intensity = ensure_dataframe_columns(self.df_intensity, ["Disease","Intensity"])
        else:
            self.df_intensity = pd.DataFrame(columns=["Disease","Intensity"])

        # Load or train model
        self.vectorizer = None
        self.model = None
        self._load_or_train()

    def _load_or_train(self):
        """Load saved model+vectorizer if present; otherwise train from CSV."""
        if os.path.exists(MODEL_FILE) and os.path.exists(VECTORIZER_FILE):
            try:
                self.model = joblib.load(MODEL_FILE)
                self.vectorizer = joblib.load(VECTORIZER_FILE)
                return
            except Exception:
                # fallback to retrain
                print("Warning: failed loading model files; retraining from CSV.")
        self._train_from_csv()

    def _train_from_csv(self):
        """Train TF-IDF + Logistic Regression on current CSV."""
        df = self.df_symptoms.copy()
        # Preprocess text column (do not overwrite original CSV)
        df['__proc__'] = df['Symptoms'].apply(preprocess)
        X = df['__proc__'].astype(str)
        y = df['Disease'].astype(str)

        # vectorizer min_df=1 to avoid dropping rare symptom combos
        self.vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=1)
        X_tfidf = self.vectorizer.fit_transform(X)

        self.model = LogisticRegression(max_iter=2000)
        self.model.fit(X_tfidf, y)

        # save for reuse
        joblib.dump(self.model, MODEL_FILE)
        joblib.dump(self.vectorizer, VECTORIZER_FILE)

    def predict_top3(self, inputs, threshold=0.7):
        """inputs: list of raw symptom strings. returns list of dicts per input."""
        processed = [preprocess(s) for s in inputs]
        X_tfidf = self.vectorizer.transform(processed)
        proba = self.model.predict_proba(X_tfidf)
        classes = self.model.classes_

        results = []
        for raw, proc, probs in zip(inputs, processed, proba):
            top_idx = np.argsort(probs)[::-1][:3]
            top3 = [(classes[i], float(probs[i])) for i in top_idx]
            pred = classes[top_idx[0]] if probs[top_idx[0]] >= threshold else "Uncertain"
            results.append({
                "Original Input": raw,
                "Preprocessed": proc,
                "Prediction": pred,
                "Top 3": top3
            })
        return results

    def suggest_tests(self, predictions):
        """predictions: list of dicts returned by predict_top3"""
        out = []
        for entry in predictions:
            original = entry['Original Input']
            tests_info = []
            for disease, score in entry['Top 3']:
                row = self.df_tests.loc[self.df_tests['Disease'] == disease]
                if not row.empty:
                    test_columns = [c for c in row.columns if c != 'Disease']
                    # join non-null values
                    vals = [str(v) for v in row.iloc[0][test_columns].values if pd.notna(v) and str(v).strip() != ""]
                    tests = ", ".join(vals) if vals else "No tests listed"
                else:
                    tests = "No tests found"
                tests_info.append((disease, score, tests))
            out.append({"input": original, "tests": tests_info})
        return out

    def check_intensity(self, predictions):
        """Return intensity info (Emergency etc.) for each disease in top3."""
        out = []
        for entry in predictions:
            info = []
            for disease, score in entry['Top 3']:
                row = self.df_intensity.loc[self.df_intensity['Disease'] == disease]
                if not row.empty and 'Intensity' in row.columns:
                    intensity_vals = [str(v) for v in row['Intensity'].values if pd.notna(v)]
                    intensity = ", ".join(intensity_vals)
                else:
                    intensity = "Unknown"
                info.append((disease, score, intensity))
            out.append({"input": entry['Original Input'], "intensity": info})
        return out

    def update_with_feedback(self, symptom_text, confirmed_disease):
        """Append confirmed pair to CSV and retrain the model."""
        # Append to dataframe in memory
        new_row = {"Disease": confirmed_disease, "Symptoms": symptom_text}
        self.df_symptoms = pd.concat([self.df_symptoms, pd.DataFrame([new_row])], ignore_index=True)

        # Append to CSV (preserve column order if possible)
        # If CSV has header Disease,Symptoms we preserve that
        try:
            self.df_symptoms.to_csv(self.symptom_file, index=False)
        except Exception as e:
            print("Warning: couldn't write to CSV:", e)

        # retrain model (overwrite saved files)
        if os.path.exists(MODEL_FILE): os.remove(MODEL_FILE)
        if os.path.exists(VECTORIZER_FILE): os.remove(VECTORIZER_FILE)
        self._train_from_csv()
        print(f"✅ Learned: '{symptom_text}' -> {confirmed_disease}")

    # utility: return raw dataframes head for inspection
    def show_data_preview(self, n=5):
        return {
            "symptoms_head": self.df_symptoms.tail(n),
            "tests_head": self.df_tests.head(n),
            "intensity_head": self.df_intensity.head(n)
        }


In [5]:
# ==========================
# Cell 4: Emotion & Urgency Detection (SpeechBrain + fallback)
# ==========================
# Try speechbrain model if available -- otherwise fallback to text+acoustic heuristics
SB_CLASSIFIER = None
if SPEECHBRAIN_AVAILABLE:
    try:
        SB_CLASSIFIER = EncoderClassifier.from_hparams(
            source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP",
            savedir="pretrained_models/emotion_recog"
        )
        print("SpeechBrain encoder classifier loaded.")
    except Exception as e:
        print("SpeechBrain available but failed to load model; falling back.", e)
        SB_CLASSIFIER = None

# transformers sentiment pipeline (optional fallback)
SENTIMENT_PIPE = None
if TRANSFORMERS_AVAILABLE:
    try:
        SENTIMENT_PIPE = pipeline("sentiment-analysis")
    except Exception:
        SENTIMENT_PIPE = None

def acoustic_heuristics(wav_path, sr_target=16000):
    """Compute simple acoustic stats. Returns dict and urgent boolean."""
    if librosa is None:
        return {"rms": None, "zcr": None, "pitch_var": None, "urgent": False}
    y, sr = librosa.load(wav_path, sr=sr_target)
    rms = float(np.mean(librosa.feature.rms(y=y)))
    zcr = float(np.mean(librosa.feature.zero_crossing_rate(y)))
    # pitch variance may be noisy; handle exceptions
    try:
        f0, voiced_flag, voiced_probs = librosa.pyin(
            y, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'), sr=sr
        )
        pitch_var = float(np.nanvar(f0))
    except Exception:
        pitch_var = 0.0
    # heuristic thresholds (tune if needed)
    urgent = (rms is not None and rms > 0.02 and pitch_var > 200) or (zcr > 0.25 if zcr is not None else False)
    return {"rms": rms, "zcr": zcr, "pitch_var": pitch_var, "urgent": bool(urgent)}

def detect_emotion_and_urgency(wav_path, transcript=None):
    """
    Returns dict:
    {
      'source': 'speechbrain'|'fallback',
      'label': str or None,
      'score': float or None,
      'acoustic': {...},
      'urgent': bool
    }
    """
    result = {"source": None, "label": None, "score": None, "acoustic": None, "urgent": False}

    # 1) SpeechBrain if available
    if SB_CLASSIFIER is not None:
        try:
            # speechbrain classify_file returns lists; wrapper used here
            out_prob, score, index, text_lab = SB_CLASSIFIER.classify_file(wav_path)
            label = text_lab[0] if isinstance(text_lab, (list, tuple)) else text_lab
            result.update({"source": "speechbrain", "label": str(label), "score": float(score)})
            result["acoustic"] = acoustic_heuristics(wav_path)
            # emergency set:
            emergency_set = {"angry", "fearful", "surprised", "panic", "frightened"}
            result["urgent"] = (str(label).lower() in emergency_set) or result["acoustic"]["urgent"]
            return result
        except Exception as e:
            print("SpeechBrain inference error, falling back:", e)

    # 2) Fallback: use STT → text sentiment (if available) + acoustic heuristics
    result["source"] = "fallback"
    if transcript is None:
        # try speech_recognition to get transcript
        try:
            r = sr.Recognizer()
            with sr.AudioFile(wav_path) as src:
                aud = r.record(src)
            transcript = r.recognize_google(aud, language="en-IN")
        except Exception:
            transcript = None

    if transcript and SENTIMENT_PIPE is not None:
        try:
            out = SENTIMENT_PIPE(transcript)[0]
            lab = "distressed" if out['label'].upper().startswith("NEG") else "calm"
            result["label"] = lab
            result["score"] = float(out['score'])
        except Exception:
            result["label"] = None
            result["score"] = None
    else:
        result["label"] = None
        result["score"] = None

    result["acoustic"] = acoustic_heuristics(wav_path)
    result["urgent"] = result["acoustic"]["urgent"] or (result["label"] == "distressed" and (result["score"] or 0) > 0.8)
    return result


c:\Users\klvns\AppData\Local\Programs\Python\Python311\Lib\site-packages\speechbrain\utils\fetching.py:151: UserWarning: Using SYMLINK strategy on Windows for fetching potentially requires elevated privileges and is not recommended. See `LocalStrategy` documentation.
  warnings.warn(
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


SpeechBrain available but failed to load model; falling back. [WinError 1314] A required privilege is not held by the client: 'C:\\Users\\klvns\\.cache\\huggingface\\hub\\models--speechbrain--emotion-recognition-wav2vec2-IEMOCAP\\snapshots\\117a9c3dff08be81a3628eecf6a66b547ec1659b\\hyperparams.yaml' -> 'd:\\srisa_c\\SIH_2025\\pretrained_models\\emotion_recog\\hyperparams.yaml'


Device set to use cpu


In [6]:
# ==========================
# Cell 5: Audio capture + helpers
# ==========================
def record_to_file(filename="temp.wav", duration=8, sample_rate=16000):
    """Record from default mic using sounddevice and save to WAV file."""
    print(f"\n Recording for {duration} seconds... speak now.")
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()
    data = np.squeeze(recording)
    # convert to int16 PCM
    int_data = np.int16(data * 32767)
    wavfile.write(filename, sample_rate, int_data)
    print(f"Saved recording to: {filename}")
    return filename

def stt_from_file(filename, language="en-IN"):
    """Speech-to-text using speech_recognition + Google (requires internet)."""
    r = sr.Recognizer()
    try:
        with sr.AudioFile(filename) as src:
            audio = r.record(src)
        text = r.recognize_google(audio, language=language)
        return text
    except sr.UnknownValueError:
        print("STT: Speech not understood.")
    except sr.RequestError as e:
        print("STT: Request error:", e)
    return None


In [11]:
# ==========================
# Cell 6: Integration / Main interactive loop
# ==========================
def run_voice_interaction(agent: SymptomAgent,
                          record_duration=6,
                          sample_rate=16000,
                          stt_language="en-IN"):
    """
    Records audio, transcribes, translates to English, gets top-3 disease predictions, 
    suggests tests, checks intensity + emotion urgency, then offers feedback learning.
    """
    # 1) Record
    wav_path = record_to_file("temp.wav", duration=record_duration, sample_rate=sample_rate)

        # 2) Speech-to-text (transcript) + auto-detect Indian language
    transcript_raw = stt_from_file(wav_path, language="hi-IN")  # default Hindi for STT
    if transcript_raw:
        print(f"\n🗣️ Transcribed text: {transcript_raw}")
    else:
        print("\n🗣️ STT failed; will attempt fallback transcription.")
        transcript_raw = ""

    # 3) Translate to English from any Indian language
    # We use src='auto' to detect source language automatically
    try:
        translated = translator.translate(transcript_raw, src='auto', dest='en').text
        print(f"🌐 Detected language → Translated to English: {translated}")
        transcript_en = translated
    except Exception as e:
        print("⚠️ Translation failed; using original text:", e)
        transcript_en = transcript_raw


    # 4) Predict disease(s)
    predictions = agent.predict_top3([transcript_en])
    for p in predictions:
        print("\n--- Prediction ---")
        print("Original Input:", p["Original Input"])
        print("Preprocessed:", p["Preprocessed"])
        print("Top 3:")
        for d, s in p["Top 3"]:
            print(f"  {d} ({s:.3f})")
        print("Main prediction:", p["Prediction"])

    # 5) Suggest tests & intensity
    tests = agent.suggest_tests(predictions)
    for t in tests:
        print("\n--- Suggested Tests ---")
        print(f"For input: {t['input']}")
        for disease, score, teststr in t['tests']:
            print(f"  {disease} ({score:.3f}) -> Tests: {teststr}")

    intens = agent.check_intensity(predictions)
    for it in intens:
        print("\n--- Intensity Info ---")
        print(f"For input: {it['input']}")
        for disease, score, intensity in it['intensity']:
            note = " 🚨 EMERGENCY" if str(intensity).lower().strip() == "emergency" else ""
            print(f"  {disease} ({score:.3f}) -> Intensity: {intensity}{note}")

    # 6) Emotion + urgency detection
    emo = detect_emotion_and_urgency(wav_path, transcript=transcript_en)
    print("\n--- Emotion / Urgency ---")
    print(f"Source: {emo['source']}")
    print(f"Label: {emo['label']}, Score: {emo['score']}")
    print("Acoustic features:", emo['acoustic'])
    print("Urgent:", emo['urgent'])

    # 7) Emergency escalation check
    emergency_by_intensity = any(str(x[2]).lower().strip() == "emergency" for x in intens[0]["intensity"])
    if emo['urgent'] or emergency_by_intensity:
        print("\n⚠️ EMERGENCY DETECTED — prioritize immediately! ⚠️")
        # optional: send alert via SMS/email/UI

    # 8) Feedback loop
    for p in predictions:
        user_confirm = input(f"\nWas the top prediction '{p['Prediction']}' correct? (yes/no/skip): ").strip().lower()
        if user_confirm == "yes":
            agent.update_with_feedback(p["Original Input"], p["Prediction"])
        elif user_confirm == "no":
            correct = input("Enter the correct disease name (exact): ").strip()
            if correct:
                agent.update_with_feedback(p["Original Input"], correct)
        else:
            print("Skipping feedback for this example.")

    print("\n✅ Interaction completed. Model persisted to disk.")


In [13]:
# ==========================
# Cell 7: Run example
# ==========================
if __name__ == "__main__":
    # initialize agent
    agent = SymptomAgent(symptom_file=SYMPTOM_FILE, test_file=TEST_FILE, intensity_file=INTENSITY_FILE)
    print("Agent ready. Data preview (tail):")
    preview = agent.show_data_preview(n=3)
    print(preview["symptoms_head"])
    # Run interactive voice session (will record from mic)
    run_voice_interaction(agent, record_duration=6)


Agent ready. Data preview (tail):
        Disease           Symptoms
1248  Uncertain  Jwalamuchi telugu
1249  Uncertain       cold and flu
1250  Uncertain    Winter and cold

 Recording for 6 seconds... speak now.
Saved recording to: temp.wav

🗣️ Transcribed text: आई हैव ए फेवर एंड कोल्ड
🌐 Detected language → Translated to English: I have a favorite and cold

--- Prediction ---
Original Input: I have a favorite and cold
Preprocessed: favorite cold
Top 3:
  Hypothyroidism (0.014)
  Uncertain (0.012)
  Cystic Fibrosis (0.006)
Main prediction: Uncertain

--- Suggested Tests ---
For input: I have a favorite and cold
  Hypothyroidism (0.014) -> Tests: No tests found
  Uncertain (0.012) -> Tests: No tests found
  Cystic Fibrosis (0.006) -> Tests: No tests found

--- Intensity Info ---
For input: I have a favorite and cold
  Hypothyroidism (0.014) -> Intensity: Unknown
  Uncertain (0.012) -> Intensity: Unknown
  Cystic Fibrosis (0.006) -> Intensity: Unknown

--- Emotion / Urgency ---
Source: 